In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000003.jpeg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000503.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000399.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000601.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000030.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000042.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000080.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000027.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000127.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000082.png
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000113.jpg
/kaggle/input/paddy-pests-dataset/paddy images/paddy without pests/00000335.jpg
/kaggle/input/paddy-pests-dataset/paddy

In [2]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="chQQqFnE0EfnTtXiohsZ")
project = rf.workspace("paddy-pest-detection2").project("paddy-pest-detection3")
version = project.version(2)
dataset = version.download("coco-segmentation")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to paddy-pest-detection3-2 in coco-segmentation:: 100%|██████████| 528/528 [00:00<00:00, 5003.49it/s]


In [4]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=aa7065cb8ab1ae6096dfe5b5726614d9df8e485438992640e69b3ae3defc9819
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [5]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, AnchorGenerator
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision import transforms as T
from PIL import Image
from pycocotools.coco import COCO
import pycocotools.mask as mask_utils
from torch.utils.data import WeightedRandomSampler
from tqdm import tqdm
from pycocotools.cocoeval import COCOeval
import utils

In [6]:
root_path=dataset.location
root_path

'/kaggle/working/paddy-pest-detection3-2'

In [7]:
# Path to COCO annotation files
TRAIN_DIR = os.path.join(root_path, 'train')
VALID_DIR = os.path.join(root_path, 'valid')
TEST_DIR = os.path.join(root_path, 'test')
TRAIN_DIR

'/kaggle/working/paddy-pest-detection3-2/train'

In [8]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [9]:
# Custom Dataset Class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(os.path.join(self.root, '_annotations.coco.json'))
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_data = self.coco.loadImgs(img_id)[0]
        path = os.path.join(self.root, img_data['file_name'])
        img = Image.open(path).convert("RGB")

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        boxes, masks, labels = [], [], []
        for ann in anns:
            bbox = ann['bbox']
            boxes.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
            labels.append(ann['category_id'])
            rles = mask_utils.frPyObjects(ann['segmentation'], img_data['height'], img_data['width'])
            masks.append(mask_utils.decode(rles).squeeze())

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        masks = torch.as_tensor(np.stack(masks), dtype=torch.uint8)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "masks": masks,
            "image_id": torch.tensor([img_id]),
            "area": (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]),
            "iscrowd": torch.zeros((len(anns),), dtype=torch.int64)
        }

        if self.transforms:
            img = self.transforms(img)
        return img, target

    def __len__(self):
        return len(self.ids)  

In [10]:
# Define transformations
# def get_transform(train):
#     transforms = [T.ToTensor()]
#     if train:
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)

def get_transform(train):
    transforms = [T.ToTensor()]
    
    if train:
        # Random Horizontal Flip
        transforms.append(T.RandomHorizontalFlip(0.5))
        
        # Random Vertical Flip
        transforms.append(T.RandomVerticalFlip(0.5))
        
        # Random Rotation
        transforms.append(T.RandomRotation(degrees=15))  # Rotate within ±15 degrees
        
        # Color Jitter
        transforms.append(T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1))
            
    return T.Compose(transforms)

In [11]:
# Initialize datasets and data loaders
dataset_train = CustomDataset(TRAIN_DIR, get_transform(train=True))
dataset_valid = CustomDataset(VALID_DIR, get_transform(train=False))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [12]:
# Dataset Balancing with Weighted Sampling
# label_counts = [0] * len(dataset_train.coco.cats)
# for idx in range(len(dataset_train)):
#     _, target = dataset_train[idx]
#     for label in target["labels"]:
#         label_counts[label] += 1

# class_weights = [1 / count if count > 0 else 0 for count in label_counts]
# sample_weights = []
# for idx in range(len(dataset_train)):
#     _, target = dataset_train[idx]
#     sample_weights.append(np.mean([class_weights[label] for label in target["labels"]]))

In [13]:
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4,shuffle=True,num_workers=2, collate_fn=lambda x: tuple(zip(*x)))
data_loader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=2, shuffle=False, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

In [14]:
# Load pre-trained Mask R-CNN model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
num_classes = len(dataset_train.coco.cats) + 1  # Including background class

# Modify the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Modify the mask predictor
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, num_classes)


# Customize anchor sizes and aspect ratios
anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,), (512,)),  # One size per feature map
    aspect_ratios=((0.5, 1.0, 2.0),) * 5          # Same ratios for all feature maps
)

model.rpn.anchor_generator = anchor_generator
model.to(device)

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:00<00:00, 186MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [15]:
# optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.0005)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [16]:
# Training loop
num_epochs = 100
#gradient_clip_value = 5.0  # Clip gradients to this value
gradient_clip_value = 1.0  # Clip gradients to this value

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    for batch_idx, (images, targets) in enumerate(data_loader_train):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        optimizer.zero_grad()
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx + 1}/{len(data_loader_train)} | Loss: {losses.item():.4f}")

    lr_scheduler.step()
    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(data_loader_train):.4f}")


Epoch 1/100
Batch 1/91 | Loss: 4.1572
Batch 11/91 | Loss: 1.0208
Batch 21/91 | Loss: 0.2952
Batch 31/91 | Loss: 0.8244
Batch 41/91 | Loss: 0.5138
Batch 51/91 | Loss: 0.2781
Batch 61/91 | Loss: 0.6886
Batch 71/91 | Loss: 0.5781
Batch 81/91 | Loss: 0.4301
Batch 91/91 | Loss: 0.5452
Epoch 1 Loss: 0.8471

Epoch 2/100
Batch 1/91 | Loss: 0.5476
Batch 11/91 | Loss: 0.4027
Batch 21/91 | Loss: 0.2440
Batch 31/91 | Loss: 0.7314
Batch 41/91 | Loss: 0.2330
Batch 51/91 | Loss: 0.1293
Batch 61/91 | Loss: 0.3655
Batch 71/91 | Loss: 0.6059
Batch 81/91 | Loss: 0.3461
Batch 91/91 | Loss: 0.4521
Epoch 2 Loss: 0.4771

Epoch 3/100
Batch 1/91 | Loss: 0.7026
Batch 11/91 | Loss: 0.5049
Batch 21/91 | Loss: 0.4289
Batch 31/91 | Loss: 0.9492
Batch 41/91 | Loss: 0.4436
Batch 51/91 | Loss: 0.7159
Batch 61/91 | Loss: 0.4428
Batch 71/91 | Loss: 0.2636
Batch 81/91 | Loss: 0.8952
Batch 91/91 | Loss: 0.5403
Epoch 3 Loss: 0.4570

Epoch 4/100
Batch 1/91 | Loss: 0.0677
Batch 11/91 | Loss: 0.4508
Batch 21/91 | Loss: 0.379

In [17]:
# Evaluation

model.eval()
coco_gt = dataset_valid.coco
coco_results = []
img_id = 0 #Important to keep track of the image id

# Evaluation
model.eval()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
coco_gt = dataset_valid.coco
coco_results = []
img_id = 0

with torch.no_grad():
    for images, targets in tqdm(data_loader_valid, desc="Evaluating"):
        images = list(image.to(device) for image in images)
        outputs = model(images)

        for i, output in enumerate(outputs):
            image_id = dataset_valid.ids[img_id]
            img_id += 1
            boxes = output['boxes'].cpu().numpy()
            scores = output['scores'].cpu().numpy()
            labels = output['labels'].cpu().numpy()
            masks = output['masks'].cpu().numpy()

            # Threshold the masks to convert probabilities to binary values
            masks = (masks > 0.5).astype(np.uint8)  # Key change: Thresholding and type conversion

            # Filter out low-confidence detections (optional)
            threshold = 0.5
            high_indices = np.where(scores > threshold)[0]

            boxes = boxes[high_indices]
            scores = scores[high_indices]
            labels = labels[high_indices]
            masks = masks[high_indices]

            for j in range(len(boxes)):
                # Convert mask to RLE format (with explicit squeeze and fortranarray)
                mask = np.squeeze(masks[j], axis=0) # Squeeze the first dimension
                fortran_mask = np.asfortranarray(mask)
                rle = mask_utils.encode(fortran_mask)
                coco_results.append({
                    'image_id': image_id,
                    'category_id': int(labels[j]),
                    'bbox': boxes[j].tolist(),
                    'score': float(scores[j]),
                    'segmentation': rle
                })
coco_results

Evaluating: 100%|██████████| 52/52 [00:13<00:00,  3.79it/s]


[{'image_id': 0,
  'category_id': 2,
  'bbox': [14.17380428314209, 2.5997560024261475, 634.5455932617188, 640.0],
  'score': 0.9125470519065857,
  'segmentation': {'size': [640, 640],
   'counts': b'XQ:b51dNh1Q2CQ=ZO4M2N2N2N2N2N2O1N10000000000000000000000000000000001O0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001O000000O100000000000000000000000000000000000000000000000000000000001O00000000000000000000000000000000000000000000000000000000000000O100001O00000000000000000000000000000000000000000000000000000000000000000000000O10000000000000000000000000000000000000000000001O000000000000000000000000O100000000000000000000000000000000000000000000001O0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000O10000000000000000001O0000000000000000000000000000000000000000000000O10000000000000001O00000O100000000000000001O00000000000000000000000000000000000000000000O

In [18]:
# Evaluate using COCOeval
coco_dt = coco_gt.loadRes(coco_results)
coco_eval = COCOeval(coco_gt, coco_dt, 'segm')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

print("mAP (mask): ", coco_eval.stats[0])
print("AP @ IoU=0.5 (mask): ", coco_eval.stats[1])
print("AP @ IoU=0.75 (mask): ", coco_eval.stats[2])
print("AR @ max detections=100 (mask): ", coco_eval.stats[8])

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.515
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.380
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.390
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.408
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1